In [1]:
import dotenv
import os
import re
import openai
dotenv.load_dotenv("./.env")
openai.api_key = os.getenv("OPENAI_API_KEY")
from tqdm import tqdm

In [2]:
from utils import split_context_to_list
import utils

[nltk_data] Downloading package punkt to /Users/nelsonlin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## (1) Gathering Knowledge

reference: https://www.sc.com/hk/credit-cards/smart/

In [3]:
document_1 = """

Smart Card 3-month Interest-free Instalment Remarks

^For spending amounts that are not eligible to earn CashBack (e.g. bill payment transactions made through Standard Chartered Online Banking), the Bank will also deduct HKD1 CashBack for every HKD180 instalment amount applied. Where there is insufficient CashBack balance on the 360° Rewards Redemption Platform for deduction, a negative balance will be displayed.
8% CashBack at Klook

From now till 31 March 2023, enjoy 8% CashBack at Klook with Smart Card. No registration and minimum spending requirement.
TOTAL PERCENTAGE OF CASHBACK EARNED FOR EACH REWARD CATEGORY
	
OFFER PERIOD
	
COMPONENTS FOR TOTAL PERCENTAGE OF CASHBACK EARNED
	
TOTAL MAXIMUM CASHBACK EARNED DURING ENTIRE PROMOTION PERIOD
8% Eligible Merchant Transactions made at Klook 	1 February 2023 to 31 March 2023 	3% under Klook offer +5% under DesignatedMerchants of SmartCard Rewards Scheme 	HKD800
Note

A. Details of the offer is as follows:

B. A maximum CashBack of HKD800 can be earned per card account under the Klook 8% CashBack Offer within the promotion period.

C. No registration, no minimum spending required and available every day.

D. For more details, please refer to Standard Chartered Smart Credit Card 8% CashBack Klook Offer Terms and Conditions.
Smart Card Welcome Offer

    From 28 February 2023 to 29 June 2023.
    New Cardholders are applicants who do not currently hold and have not cancelled any principal card of Standard Chartered Credit Card or MANHATTAN Credit Card issued by Standard Chartered Bank (Hong Kong) Limited (the “Bank”) in the past 6 months from the date of approval of their current applications for a principal card of the Standard Chartered Credit Cards.
    “Eligible Transactions” are retail purchases (including both local and overseas transactions), online purchase or posted amount of instalment purchase but do not include without limitation cash advances, gambling tokens, insurance payment, bill payment (including but without limitation tax and utilities payment) via Internet/ATMs/Phone Banking Services or other available means, any money/electronic money transfer (including but not limited to any transfer made via person to person (P2P) payment services or mobile device/app/electronic funds transfer platform), unposted/cancelled/refunded/falsified/unauthorized transactions, any transfer/top up transaction from eligible cards to any account designed by the Bank from time to time, including but not limited to Octopus O!ePay, Alipay account and other digital payment account as may be made available by the Bank from time to time.
    For the terms and conditions of the Welcome Offer, please click here.

*Smart Card Features

Ⓐ Promotion period: From 1 January to 31 December 2023

Ⓑ Upon successfully signing up for and issued with a Standard Chartered Smart Credit Card, the Bank will rebate a certain percentage of the amount of the Eligible Transactions in the form of CashBack to the card account as shown in the list below.

Ⓒ Eligible Transactions made at below Designated Merchants can earn HKD1 CashBack for every HKD20 spent (5% CashBack percentage)

    759 STORE
    Circle K Convenience Store
    HKTVmall retail stores & eShop
    Japan Home Centre retail stores & eShop
    Klook
    McDonald’s® & McDonald’s App
    PARKnSHOP, its designated retail stores (FUSION, TASTE, INTERNATIONAL, food le parc, GOURMET, GREAT FOOD HALL, EXPRESS and PARKnSHOP Frozen Food Stores outlets) & PARKnSHOP eShop
    Sa Sa retail stores & eShop
    Watsons retail stores & eShop
    Online video streaming platforms: Amazon Prime Video, Disney+, Netflix, Now TV
    Online music streaming platforms: JOOX, KKBOX, MOOV, Spotify

Ⓓ Other eligible transactions can earn HKD1 CashBack for every HKD180 spent (0.56% Cashback Percentage)

Ⓔ The total maximum amount of Eligible Transactions made at Designated Merchants that qualifies for the 5% CashBack Percentage is HKD60,000 per Card account within the promotion period. Eligible Transactions made at Designated Merchants must be completed and posted on or before 31 December 2023 based on the transaction dates as shown on the credit card monthly statement, and the Designated Merchants’ name (as determined at the Bank’s absolute discretion) must be reflected as part of the transaction descriptions on such credit card monthly statement. Any amount in excess of the HKD60,000 cap will be counted as Other Eligible Transactions.

Ⓕ For more details, please refer to Standard Chartered Smart Credit Card Rewards Scheme Terms and Conditions: https://av.sc.com/hk/zh/content/docs/hk-promo-smart-tnc.pdf.

"""

In [4]:
document_2  = """

Year-round Offer
3-month 0% Interest Instalment

Stress-free spending, split your credit card bills with 3-month interest-free instalment
Massive 5% Rebate

On designated everyday merchants. No registration, no minimum spending, available every day*.
Fees, Goodbye!

No foreign exchange fees, no annual fees, no cash advance fees. Ever.
"""

In [5]:
document_3 = """
Credit Card Welcome offer HKD1,200

    From 28 February 2023 to 29 June 2023, New Cardholders² can enjoy HK$1,200 CashBack upon making Accumulated Eligible Transactions³ of HKD4,000 within the first month from the date of issuance.
"""

In [6]:
document_4 = """

8% CashBack at Klook

    From now till 31 March 2023, enjoy 8% CashBack at Klook with Smart Card. No registration and minimum spending requirement.

    Learn More about the Smart Card Klook Offer


"""

In [7]:
document_5 = """

3-month interest free instalment

    No handling fee, no interest, no instalment quota

    Enjoy shopping with easy repayments! In just a few clicks, you can split your credit card bills in 3 months, through instalments with no interest and no handling fees, for retail spending over HK$500 or above.

    Example:
    Card spending: $9,000
    Instalment Tenor: 3 months
    Monthly repayment = $9,000/3 = $3,000
    Total repayment amount = $9,000

    Note: The Bank will deduct HKD1 CashBack on the 360° Rewards Redemption Platform for every HKD180 instalment amount applied. Example: For an instalment amount of HKD9,000, the Bank will deduct HKD50 CashBack.^

    If you select tenor of 6 months or above, the handling fee is as low as 0.15%. Click to learn more for our Statement Instalment Plan: https://www.sc.com/hk/credit-cards/statement-instalment-plan/.

"""

In [8]:
documents = [document_1,document_2,document_3,document_4,document_5]


## (2) Split documents into chunks

In [9]:
document_name = "standard chartered bank smart credit card"

In [10]:
def post_process(sentence):
    sentence = sentence.strip()
    sentence = f"These are the facts about {document_name}: {sentence}"
    return sentence

In [11]:
document_list = [document_1,document_2,document_3,document_4,document_5]

In [12]:
knowldege = []
for document in document_list:
    sentences = split_context_to_list(document_1)
    _ = [knowldege.append(post_process(s)) for s in sentences]

In [13]:
"""Design the prompt"""
def generate_prompt(context):

    question = f"I've shared some facts about {document_name} with you."+"""Please generate so many as possible pairs of questions and answers drived from facts.Please restricly follow this format 1) Q:{question},A:{answer};2)Q:{question},A:{answer}..."""

    messages = [
        {"role": "system", "content": "You are a helpful assistant and able to find the questions that user may be interested in."},
        {"role": "user", "content": context}]

    messages.append({"role": "assistant", "content": question})

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages
    )

    # return response

    answer = response['choices'][0]['message']['content']
    return answer

## (3) Generate Qustions and Answers For Training

In [15]:
prompt_generated_list = []
for context in tqdm(knowldege):
    prompt_generated = generate_prompt(context)
    prompt_generated_list.append(prompt_generated)

In [86]:
context_list = knowldege[:len(prompt_generated_list)]

In [90]:
utils.save_json(context_list, "context_list.json")

In [91]:
utils.save_json(prompt_generated_list, "prompt_generated_list.json")

## (4) Prepare data for training

In [108]:
tuning_dataset = []
for response,context in zip(prompt_generated_list,context_list):
    questions = re.findall("Q:(.+\?)",response)
    questions = [q.strip()  for q in questions]
    answers = re.findall("A:(.+\.)",response)
    answers = [a.strip() for a in answers]
    for q,a in zip(questions,answers):
        item_dict = {"instruction":q,'input':"Answer questions about {document_name}: "+context,"output":a}
        tuning_dataset.append(item_dict)
